- Criar um dataframe contendo informações de risco de roubo
- Avaliar o risco da estação(baixo, medio, alto)
    score < 500 = low
    score > 500 & < 700 = medium
    score > 700 = high
- Gravar o dataframe em formato csv

In [ ]:
file_path="/home/jovyan/work/data/202212_divvy_tripdata.csv"
station_score_path = "/home/jovyan/work/data/station_scores.csv"


from pyspark.sql import SparkSession
from pyspark.sql.functions import when

spark = SparkSession.builder \
    .appName("aula-07") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

df_trip = spark.read.option("header", "true").csv(file_path)

df_bike_share = df_trip.withColumn('end_station_name', when(df_trip['end_station_name'].isNull(), 'unknown').otherwise(df_trip['end_station_name']))

In [ ]:
df_station_score = spark.read.option("header", "true").csv(station_score_path)
display(df_station_score)

In [ ]:
df_station_scores = df_bike_share.join(df_station_score, on="end_station_id", how="inner")
display(df_station_scores)

In [ ]:
df_station_scores.withColumn("risk",
                                when((df_station_scores["score"] < 500), "low")
                                .when((df_station_scores["score"] > 500) & (df_station_scores["score"] < 700), "medium")
                                .when((df_station_scores["score"] > 700), "high")
                            )

In [8]:
output_path = "/home/jovyan/work/data/output/bike_station_risk"
df_station_scores.repartition(1).write.parquet(output_path, mode="overwrite")

In [ ]:
#https://parquet.apache.org/